<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-проекта" data-toc-modified-id="Описание-проекта-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание проекта</a></span><ul class="toc-item"><li><span><a href="#Описание-данных" data-toc-modified-id="Описание-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Описание данных</a></span></li><li><span><a href="#Общий-датасет" data-toc-modified-id="Общий-датасет-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Общий датасет</a></span></li><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Target" data-toc-modified-id="Target-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Target</a></span></li><li><span><a href="#Features" data-toc-modified-id="Features-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Features</a></span></li><li><span><a href="#Общая-таблица" data-toc-modified-id="Общая-таблица-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общая таблица</a></span><ul class="toc-item"><li><span><a href="#Объединение-датасетов" data-toc-modified-id="Объединение-датасетов-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Объединение датасетов</a></span></li><li><span><a href="#Кодирование-категориальных-признаков" data-toc-modified-id="Кодирование-категориальных-признаков-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Кодирование категориальных признаков</a></span></li></ul></li></ul></div>

## Описание проекта
Сервис  “Мой Чемпион” помогает спортивным школам фигурного катания, тренерам мониторить результаты своих подопечных и планировать дальнейшее развитие спортсменов.


**Цель**

Создать модель, помогающую находить элементы, которые могут быть успешно исполнены спортсменом на соревновании. 
Сервис будет прогнозировать прогресс и возможное выполнение тех или иных элементов программы по истории предыдущих выступлений и выполнения элементов на соревнованиях.

**Задачи DS**

Объединить данные, провести анализ данных, изучить аномалии, корреляции, описать наблюдения и сделать выводы.
Агрегировать датасет и подготовить обучающие признаки.
Создать модель для предсказания вероятности успешного выполнения элементов.
Проанализировать признаки, выявить ошибки и слабые стороны модели.

In [2]:
# Импорт библиотек

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import re

from sklearn.preprocessing import MinMaxScaler, LabelEncoder

In [6]:
from sklearn.model_selection import train_test_split, cross_val_score,  RandomizedSearchCV
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import make_scorer, f1_score


In [4]:
from sklearn.dummy import DummyClassifier

In [5]:
import catboost
import time

from tqdm import tqdm
from catboost import CatBoostClassifier, cv

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [40]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Описание данных

In [8]:
# Загрузка файла с данными
total_scores = pd.read_csv('data_01_predict_progress/total_scores.csv')
tournament_scores = pd.read_csv('data_01_predict_progress/tournament_scores.csv')
tournaments = pd.read_csv('data_01_predict_progress/tournaments.csv')
units = pd.read_csv('data_01_predict_progress/units.csv')


In [9]:
units

,id,color,school_id
0,9474,green,244.0
1,733,green,203.0
2,734,green,235.0
3,735,green,168.0
4,736,green,168.0
...,...,...,...
4591,10122,green,198.0
4592,5108,green,62.0
4593,5109,green,27.0
4594,5110,green,62.0


In [10]:
units.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4596 entries, 0 to 4595
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         4596 non-null   int64  
 1   color      4595 non-null   object 
 2   school_id  4007 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 107.8+ KB


In [11]:
units['color'].unique()

array(['green', 'lime', nan], dtype=object)

In [12]:
units['color'].value_counts()

color
green    3800
lime      795
Name: count, dtype: int64

In [13]:
units['school_id'].value_counts()

school_id
62.0     348
198.0    345
111.0    255
38.0     241
192.0    124
        ... 
2.0        1
267.0      1
163.0      1
191.0      1
227.0      1
Name: count, Length: 239, dtype: int64

**units** - 
Юнит – спортивная единица. Мы работаем с одиночным катанием, в одном юните только один спортсмен. Если бы были пары/команды, то было б несколько спортсменов в юните.
- id: идентификатор юнита
- color: категория
- school_id: идентификатор школы

В датесете содержатся пропуски в столбцах 'school_id' и 'color' 

In [15]:
tournaments.head()

,id,date_start,date_end,origin_id
0,1,2090-11-29,2090-12-01,2.0
1,2,2091-03-06,2091-03-10,1.0
2,3,2090-10-05,2090-10-08,2.0
3,4,2090-10-18,2090-10-21,2.0
4,5,2090-10-21,2090-10-24,2.0


In [17]:
tournaments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          142 non-null    int64  
 1   date_start  142 non-null    object 
 2   date_end    142 non-null    object 
 3   origin_id   142 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.6+ KB


**tournaments** -
Турнир состоит из нескольких категорий, оценки по категориям расписаны в total_scores
- id: идентификатор турнира
- date_start: дата начала
- date_end: дата завершения
- origin_id: место проведения

In [18]:
total_scores.head()

,id,unit_id,tournament_id,base_score,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str
0,442027,304,4785,47.2,43.47,102.70,59.23,0.0,17,1,Короткая программа,x Надбавка за прыжки во второй половине програ...,2,293.74,2
1,442028,604,4785,45.4,46.71,101.19,54.48,0.0,18,2,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,1,294.75,1
2,442029,409,4785,44.0,46.82,99.20,52.38,0.0,6,3,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,4,285.57,4
3,442030,524,4785,46.2,44.86,91.84,47.98,-1.0,3,4,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,3,292.42,3
4,442031,412,4785,44.2,42.59,89.73,48.14,-1.0,15,5,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,6,265.34,6


In [19]:
total_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21301 entries, 0 to 21300
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   21301 non-null  int64  
 1   unit_id              21301 non-null  int64  
 2   tournament_id        21301 non-null  int64  
 3   base_score           21301 non-null  float64
 4   components_score     21301 non-null  float64
 5   total_score          21301 non-null  float64
 6   elements_score       21301 non-null  float64
 7   decreasings_score    21301 non-null  float64
 8   starting_place       21301 non-null  int64  
 9   place                21301 non-null  int64  
 10  segment_name         21284 non-null  object 
 11  info                 20720 non-null  object 
 12  overall_place        21301 non-null  int64  
 13  overall_total_score  21284 non-null  float64
 14  overall_place_str    10814 non-null  object 
dtypes: float64(6), int64(6), object(3)
m

**total_scores** - Оценки за выступления по категориям и общие за турнир
- id: идентификатор выступления, джойнится с tournament_scores.total_score_id
- unit_id: идентификатор юнита, ключ к units.id
- tournament_id: идентификатор турнира, tournaments.id
- components_score: артистизм (мастерство, композиция, хореография)
- base_score: базовая оценка за элементы в выступлении (идеал)
- elements_score: реальная оценка всех выполненных элементов, base_score+goe
- decreasings_score: снижения оценок за ошибки
- total_score: components_score+elements_score+decreasings_score за выступление
- starting_place: жеребъёвка
- place: занятое место в категории category_name+segment_name
- segment_name: название сегмента
- info: комментарии и пояснения к оценке
- overall_place: итоговое место в турнире
- overall_total_score: итоговая оценка за весь турнир
- overall_place_str: комментарии, пояснения

In [20]:
tournament_scores.head()

,id,total_score_id,title,decrease,base_score,goe,avg_score
0,1,1,2A,NaN,3.3,0.66,3.96
1,2,1,3F+3Lo,NaN,10.2,-0.11,10.09
2,3,1,3Lz,NaN,0.0,-1.30,5.19
3,4,1,CCoSp4,NaN,3.5,0.56,4.06
4,5,1,FCSp4,NaN,3.2,0.64,3.84


In [26]:
# выполняемые элементы
tournament_scores['title'].value_counts().head(100)

title
CCoSp4        8687
2A            8500
ChSq1         8301
CCoSp3        6064
StSq2         5097
              ... 
2A+1Eu+3S      292
2Lo+1A+SЕQ     286
3S<+2T         285
1F+1Lo         278
4T             277
Name: count, Length: 100, dtype: int64

In [28]:
tournament_scores['title'].nunique()

3425

In [27]:
tournament_scores['decrease'].unique()

array([nan, 'q', '<', '!', '<<', 'nS', 'F', '*', 'e', 'nU', 'nC', '<<*',
       '<*', 'B', 'nS*', '!F', 'F*', '!*', 'q*', 'e*', 'nF', 'nB', 'B.',
       'f*', 'В', '<F', 'qF', 'b', '!<', 'f', '!F*', 'FnU', 'FF'],
      dtype=object)

- '*' - элемент не засчитывается
- q - недокручена четверть
- < - недокручено 90-180 градусов
- << - недокручено больше 180 градусов
- e  - отталкивание не с того ребра
- ! - толчок не с точного ребра
- F - упал
- B - bonus 
- nU, nC, nS, nF, nB - ошибки во вращениях

**tournament_scores** - Таблица с оценками поэлементно
- id: идентификатор оценки за конкретный элемент/комбинацию
- total_score_id: идентикатор выступления, ключ total_scores.id
- title: запись элемента или комбинации элементов с отметками об ошибках
- decrease: за что снижена оценка
- base_score: базовая оценка (идеал, цена данного элемента/комбинации, сложность)
- goe: Grade of Execute, качество исполнения, судейские надбавки/убавки
- avg_score: оценка за элемент/комбинацию (усредненная по судьям)

In [29]:
tournament_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172158 entries, 0 to 172157
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              172158 non-null  int64  
 1   total_score_id  172158 non-null  int64  
 2   title           172158 non-null  object 
 3   decrease        41185 non-null   object 
 4   base_score      172158 non-null  float64
 5   goe             172158 non-null  float64
 6   avg_score       172158 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 9.2+ MB


В 'decrease'  много пропусков - это связано с тем, что замечаней по выполнению элементов не было

В 'base_score'  - базовая оценка много полей заполненных нулями - исправим это, рассчитав 'base_score' как 'avg_score'-'goe'

In [30]:
tournament_scores['base_score'] = tournament_scores['avg_score'] - tournament_scores['goe']

In [31]:
tournament_scores.head(20)

,id,total_score_id,title,decrease,base_score,goe,avg_score
0,1,1,2A,NaN,3.30,0.66,3.96
1,2,1,3F+3Lo,NaN,10.20,-0.11,10.09
2,3,1,3Lz,NaN,6.49,-1.30,5.19
3,4,1,CCoSp4,NaN,3.50,0.56,4.06
4,5,1,FCSp4,NaN,3.20,0.64,3.84
5,6,1,LSp4,NaN,2.70,0.49,3.19
6,7,1,StSq2,NaN,2.60,0.26,2.86
7,8,8,2A,NaN,3.30,0.53,3.83
8,9,8,3F,NaN,5.83,0.42,6.25
9,10,8,3Fq+3Loq,q,10.20,-1.91,8.29


###  Общий датасет
Объединим таблицы: units и total_scores

In [32]:
# объединим units и total_scores по ключу unit_id
df = pd.merge(
    units, total_scores,
    left_on='id',
    right_on='unit_id'
)

In [34]:
df.sort_values('id_x', ascending=True).head()

,id_x,color,school_id,id_y,unit_id,tournament_id,base_score,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str
12334,1,green,198.0,7556,1,33,30.61,22.81,52.67,31.86,0.0,17,13,Короткая программа,< Недокрученный прыжок x Надбавка за прыжки во...,15,140.43,NaN
12337,1,green,198.0,10342,1,52,52.46,52.74,95.61,47.87,0.0,6,14,Произвольная программа,q Прыжок приземлён в четверть < Недокрученный ...,14,151.55,NaN
12336,1,green,198.0,441807,1,4790,52.39,55.65,111.05,55.40,0.0,28,11,Произвольная программа,q Прыжок приземлён в четверть x Надбавка за пр...,10,170.40,10
12335,1,green,198.0,7576,1,33,48.03,43.79,87.76,44.97,0.0,6,15,Произвольная программа,< Недокрученный прыжок << Пониженный прыжок x ...,15,140.43,NaN
12333,1,green,198.0,6094,1,21,59.26,55.67,118.09,62.42,0.0,5,4,Произвольная программа,q Прыжок приземлён в четверть < Недокрученный ...,4,174.38,NaN


In [35]:
# удалим столбец 'id_x' дублирующий 'unit_id'
df = df.drop(["id_x"], axis=1)

In [36]:
# переименуем столбец 'id_y'  в 'total_score_id'
df = df.rename(columns = {'id_y':'total_score_id'})

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21301 entries, 0 to 21300
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   color                21301 non-null  object 
 1   school_id            20384 non-null  float64
 2   total_score_id       21301 non-null  int64  
 3   unit_id              21301 non-null  int64  
 4   tournament_id        21301 non-null  int64  
 5   base_score           21301 non-null  float64
 6   components_score     21301 non-null  float64
 7   total_score          21301 non-null  float64
 8   elements_score       21301 non-null  float64
 9   decreasings_score    21301 non-null  float64
 10  starting_place       21301 non-null  int64  
 11  place                21301 non-null  int64  
 12  segment_name         21284 non-null  object 
 13  info                 20720 non-null  object 
 14  overall_place        21301 non-null  int64  
 15  overall_total_score  21284 non-null 

Добавим таблицу tournaments по ключу 'tournament_id'

In [38]:
df = pd.merge(
    df, tournaments,
    left_on='tournament_id',
    right_on='id'
)

In [42]:
df.head()

,color,school_id,total_score_id,unit_id,tournament_id,base_score,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,id,date_start,date_end,origin_id
0,green,244.0,442068,9474,4785,31.73,33.87,68.40,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.30,18,4785,2091-12-20,2091-12-24,2.0
1,green,244.0,442094,9474,4785,46.05,61.33,95.90,40.57,-6.0,10,18,Произвольная программа,* Недопустимый элемент < Недокрученный прыжок REP Повторение прыжка не в каскаде x Надбавка за прыжки во второй половине программы (10%) F Падение в элементе,18,164.30,18,4785,2091-12-20,2091-12-24,2.0
2,green,203.0,5246,733,17,22.40,21.64,42.70,21.06,0.0,18,16,Короткая программа,< Недокрученный прыжок x Надбавка за прыжки во второй половине программы (10%),17,107.00,NaN,17,2090-12-09,2090-12-12,2.0
3,green,203.0,5271,733,17,32.98,36.93,64.30,28.37,0.0,8,18,Произвольная программа,< Недокрученный прыжок << Пониженный прыжок e Явно неправильное ребро на толчке F/Lz x Надбавка за прыжки во второй половине программы (10%),17,107.00,NaN,17,2090-12-09,2090-12-12,2.0
4,green,235.0,442856,734,4793,21.95,22.76,41.76,21.00,-2.0,14,29,Короткая программа,< Недокрученный прыжок << Пониженный прыжок x Надбавка за прыжки во второй половине программы (10%) F Падение в элементе,25,121.41,25,4793,2091-12-09,2091-12-11,2.0


In [43]:
# удалим колонку 'id' дублирующую 'tournament_id'
df = df.drop(["id"], axis=1)

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21301 entries, 0 to 21300
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   color                21301 non-null  object 
 1   school_id            20384 non-null  float64
 2   total_score_id       21301 non-null  int64  
 3   unit_id              21301 non-null  int64  
 4   tournament_id        21301 non-null  int64  
 5   base_score           21301 non-null  float64
 6   components_score     21301 non-null  float64
 7   total_score          21301 non-null  float64
 8   elements_score       21301 non-null  float64
 9   decreasings_score    21301 non-null  float64
 10  starting_place       21301 non-null  int64  
 11  place                21301 non-null  int64  
 12  segment_name         21284 non-null  object 
 13  info                 20720 non-null  object 
 14  overall_place        21301 non-null  int64  
 15  overall_total_score  21284 non-null 

Добавим таблицу tournament_scores по ключу 'total_score_id'

In [45]:
df = pd.merge(
    df, tournament_scores,
    left_on='total_score_id',
    right_on='total_score_id'
)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172158 entries, 0 to 172157
Data columns (total 26 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   color                172158 non-null  object 
 1   school_id            164913 non-null  float64
 2   total_score_id       172158 non-null  int64  
 3   unit_id              172158 non-null  int64  
 4   tournament_id        172158 non-null  int64  
 5   base_score_x         172158 non-null  float64
 6   components_score     172158 non-null  float64
 7   total_score          172158 non-null  float64
 8   elements_score       172158 non-null  float64
 9   decreasings_score    172158 non-null  float64
 10  starting_place       172158 non-null  int64  
 11  place                172158 non-null  int64  
 12  segment_name         172115 non-null  object 
 13  info                 168356 non-null  object 
 14  overall_place        172158 non-null  int64  
 15  overall_total_sco

In [47]:
#количество уникальных значений 'origin_id' -место проведения
df['origin_id'].value_counts()

origin_id
2.0    78204
1.0    55240
0.0    38714
Name: count, dtype: int64

In [48]:
print('Количество турниров:', df['tournament_id'].nunique())

Количество турниров: 142


In [49]:
print('Количество записей по каждому турниру:')
df['tournament_id'].value_counts()

Количество записей по каждому турниру:


tournament_id
6859    4314
78      4223
7114    4045
6848    3660
77      3398
7098    3041
7117    2860
85      2853
7103    2838
79      2786
7110    2769
88      2749
7106    2635
6864    2634
6866    2433
2       2363
24      2255
39      2229
76      2222
68      2218
6699    2203
6885    2011
7092    1982
18      1909
3       1885
14      1873
4992    1863
176     1850
47      1767
40      1728
7111    1724
4790    1709
4793    1638
17      1617
45      1588
48      1550
7109    1547
73      1519
398     1484
6847    1454
84      1430
4789    1412
6845    1410
6160    1403
21      1399
5879    1377
51      1337
177     1296
7115    1281
4794    1266
52      1265
23      1257
4865    1247
33      1245
1       1221
69      1180
91      1160
6850    1159
6849    1155
49      1155
7112    1116
7104    1032
72      1019
222     1009
92      1008
44      1008
263     1001
6676     999
6844     987
43       975
29       960
50       957
2616     948
86       940
22       936
95       93

In [50]:
print('Количество записей по каждому участнику:')
df["unit_id"].value_counts()

Количество записей по каждому участнику:


unit_id
351      442
350      411
354      365
310      363
313      363
        ... 
9526       2
9533       2
9532       2
9521       2
10015      1
Name: count, Length: 3393, dtype: int64

In [51]:
print('Количество турниров для каждого участника')
display(df.groupby('unit_id')['tournament_id'].nunique())

Количество турниров для каждого участника


unit_id
1        15
3        12
4         6
5         6
6        12
         ..
35003     1
35004     1
35005     1
35006     1
35024     1
Name: tournament_id, Length: 3393, dtype: int64

В данных присутствуют спортсмены, принимавшие участие только в одном соревновании.

### Вывод: 
Нам предоставлены несколько таблиц в которых хранятся данные по каждому спортсмену: в каких турнирах участвовал, какие элементы выполнял, какие ошибки были, сколько баллов набрал, какие места в каком сегменте завоёвывал.

В данных есть информация о 3393 спортсменах. Количество турниров: 142 в трёх различных местах.

Все датасеты были соединины в один.

В данных присутствуют пропуски в столбцах: 
- 'school_id' - идентификатор школы, восстановить невозможно
- 'info' - комментарии к выступлению, восстановить невозможно
- 'segment_name' - название сегмента, восстановить невозможно
- 'overall_total_score' - итоговая оценка за весь,  турнир,восстановить невозможно
- 'overall_place_str', восстановить невозможно
- 'decrease' - за что снижена оценка. Это связано с тем, что замечаней по выполнению элементов не было. Именно исходя из этих данным будем выбирать элементы выполненные без ошибок. (таргет)


## Target

Создадим новый датасет с таргетом исходя из имеющихся данных: для каждого спортсмена выберем элементы, которые он выполнял без ошибок.

In [54]:
# Функция для извлечения элементов из строки
def extract_elements(text):
    # Паттерн для совпадения элементов, исключая строчную 'e', заглавные 'V' и 'B', и 'q' после 'A', 'T', 'F'
    pattern = r'\b\d*[A-Z][a-zA-Z]*\d*\b'
    # Находим все совпадения
    matches = re.findall(pattern, text)
    # Удаляем 'e' внутри каждого совпадения и обрабатываем случаи с V или B в конце элемента
    final_matches = []
    for match in matches:
        # Убираем 'e'
        match = re.sub(r'e', '', match)
        # Удаляем V или B в конце элемента
        match = re.sub(r'[B]$', '', match)
        match = re.sub(r'[V]$', '', match)
        if re.search(r'[ATFo]q', match):
            # Убираем "q" после "A", "T", "F"
            final_matches.append(re.sub(r'q', '', match))
        else:
            final_matches.append(match)
    return final_matches


# Применение функции к каждому элементу столбца
df['list_title'] = df['title'].apply(extract_elements)

df['list_title'] = df['list_title'].apply(lambda x: [item for item in x if item not in ['COMBO', 'REP']])

In [55]:
df.head()

,color,school_id,total_score_id,unit_id,tournament_id,base_score_x,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,date_start,date_end,origin_id,id,title,decrease,base_score_y,goe,avg_score,list_title
0,green,244.0,442068,9474,4785,31.73,33.87,68.4,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.3,18,2091-12-20,2091-12-24,2.0,156595,3Lz+3T,NaN,10.10,1.18,11.28,"[3Lz, 3T]"
1,green,244.0,442068,9474,4785,31.73,33.87,68.4,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.3,18,2091-12-20,2091-12-24,2.0,156596,2A,NaN,3.30,0.85,4.15,[2A]
2,green,244.0,442068,9474,4785,31.73,33.87,68.4,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.3,18,2091-12-20,2091-12-24,2.0,156597,FCSp4,NaN,3.20,0.87,4.07,[FCSp4]
3,green,244.0,442068,9474,4785,31.73,33.87,68.4,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.3,18,2091-12-20,2091-12-24,2.0,156598,3F,NaN,5.83,-1.36,4.47,[3F]
4,green,244.0,442068,9474,4785,31.73,33.87,68.4,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.3,18,2091-12-20,2091-12-24,2.0,156599,SSp4,NaN,2.50,0.61,3.11,[SSp4]


In [56]:
# для 'unit_id' = 1  выведем данные
df.query('unit_id == 1')

,color,school_id,total_score_id,unit_id,tournament_id,base_score_x,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,date_start,date_end,origin_id,id,title,decrease,base_score_y,goe,avg_score,list_title
100447,green,198.0,442658,1,4844,28.96,28.00,60.70,32.70,0.0,4,3,Короткая программа,! Неясное ребро на толчке F/Lz x Надбавка за прыжки во второй половине программы (10%),3,183.24,3,2091-12-25,2091-12-26,1.0,161036,2A,NaN,3.30,0.59,3.89,[2A]
100448,green,198.0,442658,1,4844,28.96,28.00,60.70,32.70,0.0,4,3,Короткая программа,! Неясное ребро на толчке F/Lz x Надбавка за прыжки во второй половине программы (10%),3,183.24,3,2091-12-25,2091-12-26,1.0,161037,3F!,!,5.30,0.11,5.41,[3F]
100449,green,198.0,442658,1,4844,28.96,28.00,60.70,32.70,0.0,4,3,Короткая программа,! Неясное ребро на толчке F/Lz x Надбавка за прыжки во второй половине программы (10%),3,183.24,3,2091-12-25,2091-12-26,1.0,161038,LSp4,NaN,2.70,0.49,3.19,[LSp4]
100450,green,198.0,442658,1,4844,28.96,28.00,60.70,32.70,0.0,4,3,Короткая программа,! Неясное ребро на толчке F/Lz x Надбавка за прыжки во второй половине программы (10%),3,183.24,3,2091-12-25,2091-12-26,1.0,161039,3Lz+2Lo,NaN,8.36,0.83,9.19,"[3Lz, 2Lo]"
100451,green,198.0,442658,1,4844,28.96,28.00,60.70,32.70,0.0,4,3,Короткая программа,! Неясное ребро на толчке F/Lz x Надбавка за прыжки во второй половине программы (10%),3,183.24,3,2091-12-25,2091-12-26,1.0,161040,FCSp4,NaN,3.20,0.64,3.84,[FCSp4]
100452,green,198.0,442658,1,4844,28.96,28.00,60.70,32.70,0.0,4,3,Короткая программа,! Неясное ребро на толчке F/Lz x Надбавка за прыжки во второй половине программы (10%),3,183.24,3,2091-12-25,2091-12-26,1.0,161041,StSq2,NaN,2.60,0.52,3.12,[StSq2]
100453,green,198.0,442658,1,4844,28.96,28.00,60.70,32.70,0.0,4,3,Короткая программа,! Неясное ребро на толчке F/Lz x Надбавка за прыжки во второй половине программы (10%),3,183.24,3,2091-12-25,2091-12-26,1.0,161042,CCoSp4,NaN,3.50,0.56,4.06,[CCoSp4]
100454,green,198.0,442666,1,4844,59.07,57.41,122.54,65.13,0.0,6,3,Произвольная программа,q Прыжок приземлён в четверть x Надбавка за прыжки во второй половине программы (10%),3,183.24,3,2091-12-25,2091-12-26,1.0,161102,3T,NaN,4.20,0.76,4.96,[3T]
100455,green,198.0,442666,1,4844,59.07,57.41,122.54,65.13,0.0,6,3,Произвольная программа,q Прыжок приземлён в четверть x Надбавка за прыжки во второй половине программы (10%),3,183.24,3,2091-12-25,2091-12-26,1.0,161103,3Lz+2T,NaN,7.20,0.83,8.03,"[3Lz, 2T]"
100456,green,198.0,442666,1,4844,59.07,57.41,122.54,65.13,0.0,6,3,Произвольная программа,q Прыжок приземлён в четверть x Надбавка за прыжки во второй половине программы (10%),3,183.24,3,2091-12-25,2091-12-26,1.0,161104,2A,NaN,3.30,0.59,3.89,[2A]


In [57]:
# cписок элементов
elements = ["USp", "LSp", "LSp1",'LSp2', 'LSp3', 'LSp4',
                "CSp", "CSp1", "CSp2", "CSp3", "CSp4",
                "SSp", "SSp1",  "SSp2", "SSp3", "SSp4",
                "FUSp",
                "FLSp1", 
                "FCSp", "FCSp1", "FCSp2", "FCSp3", "FCSp4",
                "FSSp", "FSSp1", "FSSp2", "FSSp3", "FSSp4", 
                "CUSp", 
                "CLSp",
                "CCSp", "CCSp1", "CCSp2", "CCSp3", "CCSp4", 
                "CSSp", "CSSp1", "CSSp2", "CSSp3", "CSSp4", 
                "CoSp", "CoSp1", "CoSp2", 
                "CCoSp", "CCoSp1", "CCoSp2", "CCoSp3", "CCoSp4", 
                "StSq", "StSq1", "StSq2", "StSq3", "StSq4", 
                "ChSq", "ChSq1",
                "T", "1T", "2T", "3T", "4T", 
                "S", "1S", "2S", "3S", "4S", 
                "Lo", "1Lo", "2Lo", "3Lo", "4Lo", 
                "F", "1F", "2F", "3F", "4F", 
                "Lz", "1Lz", "2Lz", "3Lz", "4Lz", 
                "A", "1A", "2A", "3A", "4A",
                "1Eu"]

In [58]:
# Создание списка для накопления данных
rows = []

# Заполнение таблицы
for unit_id in df['unit_id'].unique():
    # Получаем все записи для данного unit_id
    unit_records = df[df['unit_id'] == unit_id]
    decrease_value = unit_records['decrease'].iloc[0]  # Берем значение decrease из первой записи
    
    # Инициализация словаря для данного unit_id
    unit_dict = {'unit_id': unit_id}
    for target in elements:
        # Если 'decrease' равно NaN, устанавливаем значение 1 для всех таргетов
        if pd.isna(decrease_value):
            unit_dict[target] = 1 if any(target in lst for lst in unit_records['list_title']) else 0
        # Если 'decrease' не равно NaN, устанавливаем значение 0 для всех фичей
        else:
            unit_dict[target] = 0
    
    # Добавляем словарь в список
    rows.append(unit_dict)

# Преобразование списка словарей в DataFrame
target_df = pd.DataFrame(rows)

In [59]:
target_df.head(10)

,unit_id,USp,LSp,LSp1,LSp2,LSp3,LSp4,CSp,CSp1,CSp2,CSp3,CSp4,SSp,SSp1,SSp2,SSp3,SSp4,FUSp,FLSp1,FCSp,FCSp1,FCSp2,FCSp3,FCSp4,FSSp,FSSp1,FSSp2,FSSp3,FSSp4,CUSp,CLSp,CCSp,CCSp1,CCSp2,CCSp3,CCSp4,CSSp,CSSp1,CSSp2,CSSp3,CSSp4,CoSp,CoSp1,CoSp2,CCoSp,CCoSp1,CCoSp2,CCoSp3,CCoSp4,StSq,StSq1,StSq2,StSq3,StSq4,ChSq,ChSq1,T,1T,2T,3T,4T,S,1S,2S,3S,4S,Lo,1Lo,2Lo,3Lo,4Lo,F,1F,2F,3F,4F,Lz,1Lz,2Lz,3Lz,4Lz,A,1A,2A,3A,4A,1Eu
0,9474,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,1
1,733,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
2,734,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,735,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0
4,736,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,737,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,1,1,1,0,0,0,1,1,0,0,0,1,0,0,1
6,738,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1
7,739,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,740,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0
9,1007,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [60]:
#объединим столбцы с одинаковыми по смыслу элементами: 
target_df['LSp1'] = target_df[['LSp', 'LSp1']].max(axis=1)
target_df['CSp1'] = target_df[['CSp', 'CSp1']].max(axis=1) 
target_df['SSp1'] = target_df[['SSp', 'SSp1']].max(axis=1)
target_df['FCSp1'] = target_df[['FCSp', 'FCSp1']].max(axis=1)
target_df['FSSp1'] = target_df[['FSSp', 'FSSp1']].max(axis=1)
target_df['CCSp1'] = target_df[['CCSp', 'CCSp1']].max(axis=1)
target_df['CSSp1'] = target_df[['CSSp', 'CSSp1']].max(axis=1)
target_df['CoSp1'] = target_df[['CoSp', 'CoSp1']].max(axis=1)
target_df['CCoSp1'] = target_df[['CCoSp', 'CCoSp1']].max(axis=1)
target_df['StSq1'] = target_df[['StSq', 'StSq1']].max(axis=1)
target_df['ChSq'] = target_df[['ChSq', 'ChSq1']].max(axis=1)
target_df['1T'] = target_df[['T', '1T']].max(axis=1)
target_df['1S'] = target_df[['S', '1S']].max(axis=1)
target_df['1Lo'] = target_df[['Lo', '1Lo']].max(axis=1)
target_df['1F'] = target_df[['F', '1F']].max(axis=1)
target_df['1Lz'] = target_df[['Lz', '1Lz']].max(axis=1) 
target_df['1A'] = target_df[['A', '1A']].max(axis=1)

In [62]:
target_df = target_df.drop(columns=['LSp', 'CSp', 'SSp', 'FCSp', 'FSSp', 'CCSp', 'CSSp', 'CoSp',
                                       'CCoSp', 'StSq', 'ChSq1', 'T', 'S', 'Lo', 'F', 'Lz', 'A' ])

In [63]:
target_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3393 entries, 0 to 3392
Data columns (total 70 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   unit_id  3393 non-null   int64
 1   USp      3393 non-null   int64
 2   LSp1     3393 non-null   int64
 3   LSp2     3393 non-null   int64
 4   LSp3     3393 non-null   int64
 5   LSp4     3393 non-null   int64
 6   CSp1     3393 non-null   int64
 7   CSp2     3393 non-null   int64
 8   CSp3     3393 non-null   int64
 9   CSp4     3393 non-null   int64
 10  SSp1     3393 non-null   int64
 11  SSp2     3393 non-null   int64
 12  SSp3     3393 non-null   int64
 13  SSp4     3393 non-null   int64
 14  FUSp     3393 non-null   int64
 15  FLSp1    3393 non-null   int64
 16  FCSp1    3393 non-null   int64
 17  FCSp2    3393 non-null   int64
 18  FCSp3    3393 non-null   int64
 19  FCSp4    3393 non-null   int64
 20  FSSp1    3393 non-null   int64
 21  FSSp2    3393 non-null   int64
 22  FSSp3    3393 non-null  

In [64]:
target_df.head()

,unit_id,USp,LSp1,LSp2,LSp3,LSp4,CSp1,CSp2,CSp3,CSp4,SSp1,SSp2,SSp3,SSp4,FUSp,FLSp1,FCSp1,FCSp2,FCSp3,FCSp4,FSSp1,FSSp2,FSSp3,FSSp4,CUSp,CLSp,CCSp1,CCSp2,CCSp3,CCSp4,CSSp1,CSSp2,CSSp3,CSSp4,CoSp1,CoSp2,CCoSp1,CCoSp2,CCoSp3,CCoSp4,StSq1,StSq2,StSq3,StSq4,ChSq,1T,2T,3T,4T,1S,2S,3S,4S,1Lo,2Lo,3Lo,4Lo,1F,2F,3F,4F,1Lz,2Lz,3Lz,4Lz,1A,2A,3A,4A,1Eu
0,9474,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,1,0,1,0,1,1,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,1,0,0,1,0,0,1
1,733,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
2,734,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,735,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0
4,736,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Features

Создадим новую таблицу с фичами исходя из имеющихся данных.
Для каждого спортсмены определим
- количество турниров, в которых он участвовал
- даты первого и последнего турниров
- категория ('color')


**Количество турниров, в которых участвовал спортсмен**

In [65]:
feature_df = df.groupby('unit_id')['tournament_id'].nunique().reset_index()

In [67]:
feature_df.rename(columns={'tournament_id': 'count_tournament'}, inplace=True)

In [68]:
feature_df

,unit_id,count_tournament
0,1,15
1,3,12
2,4,6
3,5,6
4,6,12
...,...,...
3388,35003,1
3389,35004,1
3390,35005,1
3391,35006,1


**Даты первого и последнего турнира**

In [69]:
# Преобразование столбца 'date_start' в формат datetime, обработка ошибок
df['date_start'] = pd.to_datetime(df['date_start'], errors='coerce')

In [71]:
# первый турнир
first_tournament = df.groupby('unit_id')['date_start'].min().reset_index()
first_tournament.rename(columns={'date_start': 'first_tournament'}, inplace=True)
first_tournament

,unit_id,first_tournament
0,1,2090-04-19
1,3,2090-10-05
2,4,2090-10-26
3,5,2090-10-26
4,6,2090-10-26
...,...,...
3388,35003,2092-04-29
3389,35004,2092-04-29
3390,35005,2092-04-29
3391,35006,2092-04-29


In [72]:
feature_df = pd.merge(
    feature_df, first_tournament,
    left_on='unit_id',
    right_on='unit_id'
)

In [74]:
feature_df.head()

,unit_id,count_tournament,first_tournament
0,1,15,2090-04-19
1,3,12,2090-10-05
2,4,6,2090-10-26
3,5,6,2090-10-26
4,6,12,2090-10-26


In [75]:
# последний турнир
last_tournament = df.groupby('unit_id')['date_start'].max().reset_index()
last_tournament.rename(columns={'date_start': 'last_tournament'}, inplace=True)
feature_df = pd.merge(
    feature_df, last_tournament,
    left_on='unit_id',
    right_on='unit_id'
)

**Категория**

In [76]:
color = df.groupby('unit_id')['color'].max().reset_index()
feature_df = pd.merge(
    feature_df, color,
    left_on='unit_id',
    right_on='unit_id'
)

**Лучшее и худшее место в различных сегментах**

In [79]:
print('Список уникальных названий сегментов:', df['segment_name'].unique())

Список уникальных названий сегментов: ['Короткая программа' 'Произвольная программа' 'Кoроткая программа'
 'Прoизвольная прoграмма' 'Пpoизвольная программа' 'Элементы'
 'Кopoткaя пpoгpaммa' 'Пpoизвoльнaя пpoгpaммa' 'Произвольная программа.'
 'Прыжки' 'Элeмeнты' 'Kopoткaя пpoгpaммa' 'Произвольнaя программа'
 'Коpоткая программа' 'Пpoизвольнaя программа' 'Интерпретация.' nan
 'Интерпретация']


In [80]:
# количество записей для каждого уникального значения сегмента
df['segment_name'].value_counts()

segment_name
Произвольная программа     91935
Короткая программа         53525
Произвольная программа.    17091
Элементы                    3520
Пpoизвoльнaя пpoгpaммa      1581
Прыжки                       852
Кoроткая программа           700
Кopoткaя пpoгpaммa           659
Произвольнaя программа       473
Прoизвольная прoграмма       427
Kopoткaя пpoгpaммa           377
Элeмeнты                     341
Пpoизвольная программа       315
Коpоткая программа           126
Интерпретация.               111
Пpoизвольнaя программа        70
Интерпретация                 12
Name: count, dtype: int64

Приведём значения секментов к единому написанию

In [81]:
# переименуем значения - пиведём их к единому написанию
df['segment_name'] = df['segment_name'].replace({
    'Пpoизвoльнaя пpoгpaммa': 'Произвольная программа',
    'Произвольная программа.': 'Произвольная программа',
    'Пpoизвольнaя программа': 'Произвольная программа',
    'Пpoизвольная программа': 'Произвольная программа',
    'Прoизвольная прoграмма': 'Произвольная программа',
    'Пpoизвольнaя программа': 'Произвольная программа',
    'Произвольнaя программа': 'Произвольная программа',
    'Kopoткaя пpoгpaммa': 'Короткая программа',
    'Кopoткaя пpoгpaммa': 'Короткая программа',
    'Коpоткая программа': 'Короткая программа',
    'Кoроткая программа': 'Короткая программа',
    'Пpогpaммa': 'Программа',
    'Элeмeнты': 'Элементы',
    'Прыжки': 'Прыжки',
    'Интерпретация.': 'Интерпретация'
})

print(df['segment_name'].unique())

['Короткая программа' 'Произвольная программа' 'Элементы' 'Прыжки'
 'Интерпретация' nan]


In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172158 entries, 0 to 172157
Data columns (total 27 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   color                172158 non-null  object        
 1   school_id            164913 non-null  float64       
 2   total_score_id       172158 non-null  int64         
 3   unit_id              172158 non-null  int64         
 4   tournament_id        172158 non-null  int64         
 5   base_score_x         172158 non-null  float64       
 6   components_score     172158 non-null  float64       
 7   total_score          172158 non-null  float64       
 8   elements_score       172158 non-null  float64       
 9   decreasings_score    172158 non-null  float64       
 10  starting_place       172158 non-null  int64         
 11  place                172158 non-null  int64         
 12  segment_name         172115 non-null  object        
 13  info          

In [83]:
# удалим записи с пропусками: 43 записи
df = df.dropna(subset =['segment_name'])

In [84]:
print(df['segment_name'].unique())

['Короткая программа' 'Произвольная программа' 'Элементы' 'Прыжки'
 'Интерпретация']


In [85]:
# Группировка по 'unit_id' и 'segment_name' и нахождение минимального значения 'place'
best_place = df.groupby(['unit_id', 'segment_name'])['place'].min().unstack().reset_index()

# Переименование столбцов
best_place.columns = ['unit_id', 'best_place_interpretation', 'best_place_short', 'best_place_free', 'best_place_jump', 'best_place_elements']
best_place.head()

,unit_id,best_place_interpretation,best_place_short,best_place_free,best_place_jump,best_place_elements
0,1,NaN,1.0,3.0,NaN,NaN
1,3,NaN,1.0,1.0,NaN,NaN
2,4,NaN,2.0,2.0,NaN,NaN
3,5,NaN,2.0,2.0,NaN,NaN
4,6,NaN,3.0,1.0,NaN,NaN


In [86]:
# заполним пропуски 0
best_place = best_place.fillna(0)

In [87]:
feature_df = pd.merge(
    feature_df, best_place,
    left_on='unit_id',
    right_on='unit_id'
)

In [88]:
# Группировка по 'unit_id' и 'segment_name' и нахождение максимальное значения 'place'
worst_place = df.groupby(['unit_id', 'segment_name'])['place'].max().unstack().reset_index()

# Переименование столбцов
worst_place.columns = ['unit_id', 'worst_place_interpretation', 'worst_place_short', 'worst_place_free', 'worst_place_jump', 'worst_place_elements']

worst_place = worst_place.fillna(0)

feature_df = pd.merge(
    feature_df, worst_place,
    left_on='unit_id',
    right_on='unit_id'
)

**Лучший и худший бал за выступление по сегментам**

In [89]:
# Группировка по 'unit_id' и 'segment_name' и нахождение минимальное значения 'total_score'
worst_total_score = df.groupby(['unit_id', 'segment_name'])['total_score'].min().unstack().reset_index()

# Переименование столбцов
worst_total_score.columns = ['unit_id', 'worst_total_score_interpretation', 'worst_total_score_short', 'worst_total_score_free', 'worst_total_score_jump', 'worst_total_score_elements']

worst_total_score = worst_total_score.fillna(0)

feature_df = pd.merge(
    feature_df, worst_total_score,
    left_on='unit_id',
    right_on='unit_id'
)

In [90]:
# Группировка по 'unit_id' и 'segment_name' и нахождение максимальное значения 'total_score'
best_total_score = df.groupby(['unit_id', 'segment_name'])['total_score'].max().unstack().reset_index()

# Переименование столбцов
best_total_score.columns = ['unit_id', 'best_total_score_interpretation', 'best_total_score_short', 'best_total_score_free', 'best_total_score_jump', 'best_total_score_elements']

best_total_score = best_total_score.fillna(0)

feature_df = pd.merge(
    feature_df, best_total_score,
    left_on='unit_id',
    right_on='unit_id'
)

**Максимальное и минимальное значение  Grade of Execute - надбавки за элементы**

In [91]:
max_goe = df.groupby('unit_id')['goe'].max().reset_index()
max_goe.rename(columns={'goe': 'max_goe'}, inplace=True)
feature_df = pd.merge(
    feature_df, max_goe,
    left_on='unit_id',
    right_on='unit_id'
)

In [92]:
min_goe = df.groupby('unit_id')['goe'].min().reset_index()
min_goe.rename(columns={'goe': 'min_goe'}, inplace=True)
feature_df = pd.merge(
    feature_df, min_goe,
    left_on='unit_id',
    right_on='unit_id'
)

**Среднее значение среднего бала заэлементы**

In [94]:
avg_score = df.groupby('unit_id')['avg_score'].mean().reset_index()
feature_df = pd.merge(
    feature_df, min_avg_score,
    left_on='unit_id',
    right_on='unit_id'
)

In [95]:
feature_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3382 entries, 0 to 3381
Data columns (total 29 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   unit_id                           3382 non-null   int64         
 1   count_tournament                  3382 non-null   int64         
 2   first_tournament                  3382 non-null   datetime64[ns]
 3   last_tournament                   3382 non-null   datetime64[ns]
 4   color                             3382 non-null   object        
 5   best_place_interpretation         3382 non-null   float64       
 6   best_place_short                  3382 non-null   float64       
 7   best_place_free                   3382 non-null   float64       
 8   best_place_jump                   3382 non-null   float64       
 9   best_place_elements               3382 non-null   float64       
 10  worst_place_interpretation        3382 non-null 

## Общая таблица

### Объединение датасетов

In [97]:
# Объединим данные по каждому спортсмену в общую таблицу
full_df = pd.merge(
    feature_df, target_df,
    left_on='unit_id',
    right_on='unit_id'
)

In [98]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3382 entries, 0 to 3381
Data columns (total 98 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   unit_id                           3382 non-null   int64         
 1   count_tournament                  3382 non-null   int64         
 2   first_tournament                  3382 non-null   datetime64[ns]
 3   last_tournament                   3382 non-null   datetime64[ns]
 4   color                             3382 non-null   object        
 5   best_place_interpretation         3382 non-null   float64       
 6   best_place_short                  3382 non-null   float64       
 7   best_place_free                   3382 non-null   float64       
 8   best_place_jump                   3382 non-null   float64       
 9   best_place_elements               3382 non-null   float64       
 10  worst_place_interpretation        3382 non-null 

### Кодирование категориальных признаков

In [99]:
# закодируем категориальные признаки столбец 'color'
# Кодирование категориальных признаков с использованием LabelEncoder
label_encoders = {}
for column in ['color']:
    le = LabelEncoder()
    full_df[column] = le.fit_transform(full_df[column])
    label_encoders[column] = le
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3382 entries, 0 to 3381
Data columns (total 98 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   unit_id                           3382 non-null   int64         
 1   count_tournament                  3382 non-null   int64         
 2   first_tournament                  3382 non-null   datetime64[ns]
 3   last_tournament                   3382 non-null   datetime64[ns]
 4   color                             3382 non-null   int64         
 5   best_place_interpretation         3382 non-null   float64       
 6   best_place_short                  3382 non-null   float64       
 7   best_place_free                   3382 non-null   float64       
 8   best_place_jump                   3382 non-null   float64       
 9   best_place_elements               3382 non-null   float64       
 10  worst_place_interpretation        3382 non-null 

Уберём из датасета записи, где количество турниров меньше 2

In [100]:
# Фильтрация датасета
full_df = full_df[full_df['count_tournament'] > 1]

In [101]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2291 entries, 0 to 3333
Data columns (total 98 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   unit_id                           2291 non-null   int64         
 1   count_tournament                  2291 non-null   int64         
 2   first_tournament                  2291 non-null   datetime64[ns]
 3   last_tournament                   2291 non-null   datetime64[ns]
 4   color                             2291 non-null   int64         
 5   best_place_interpretation         2291 non-null   float64       
 6   best_place_short                  2291 non-null   float64       
 7   best_place_free                   2291 non-null   float64       
 8   best_place_jump                   2291 non-null   float64       
 9   best_place_elements               2291 non-null   float64       
 10  worst_place_interpretation        2291 non-null   flo